# k-Fold Cross Validation and Grid Search

## Importing the libraries

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [ ]:
dataset = pd.read_csv("Social_Network_Ads.csv")

#independent variables (all but the rightmost column)
X = dataset.iloc[:, :-1].values
# dependent variable (variable to be predicted; rightmost column)
y = dataset.iloc[:, -1].values

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
# Changed the test size to 25% so it's nice round numbers in a 400 entry test set

## Feature Scaling

In [ ]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test)
# The only categorical variable is the dependent variable; no rows or columns need to be excluded

## Training the SVM model on the Training set

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel="rbf", random_state=0)
classifier.fit(X_train, y_train)

## Predicting a new result

In [ ]:
print(classifier.predict(sc.transform([[30, 87000]]))) 
# It's still only one row, i.e. a 1d array
# must be scaled the same as the training data


## Predicting the Test set results

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1 )), 1)) 
# predictions on the left, real data on the right

## Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

## Applying k-Fold Cross Validation

In [ ]:
# selects multiple random test sets so you know the model didn't just get lucky on an easy test set
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator= classifier, X = X_train, y= y_train, cv= 10) # cv is the number of folds
print("Accuracy: {:.2f} %".format(accuracies.mean()*100)) # displays the mean of the accuracies as float with two decimal places
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100)) # Most of the results were +- this percentage from the mean. Results were pretty high

## Applying Grid Search to find the best model parameters

In [ ]:

from sklearn.model_selection import GridSearchCV

# Adjusts the hyperparameters of the SVM model (C, kernel, degree, gamma)
# gamma only applies to rbf kernel
parameters = [{"C": [0.25, 0.5, 0.75, 1], "kernel": ["linear"]},
            {"C": [0.25, 0.5, 0.75, 1], "kernel": ["rbf"], "gamma": [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator=classifier, 
                        param_grid=parameters,
                        scoring="accuracy",
                        cv= 10,
                        n_jobs= -1) # use all processor cores

grid_search.fit(X_train, y_train) # this is a LOT of training

best_accuracy = grid_search.best_score_ # the final underscore indicates that this is an attribute of the object, not a function
best_parameters = grid_search.best_params_

print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters) # a standard way to print a string followed by a variable


## Visualizing the Training set results

In [ ]:
from matplotlib.colors import ListedColormap
X_set, y_set = sc.inverse_transform(X_train), y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 10, stop = X_set[:, 0].max() + 10, step = 1),
                    np.arange(start = X_set[:, 1].min() - 1000, stop = X_set[:, 1].max() + 1000, step = 1))
plt.contourf(X1, X2, classifier.predict(sc.transform(np.array([X1.ravel(), X2.ravel()]).T)).reshape(X1.shape),
            alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1], c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Logistic Regression (Training set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

## Visualizing the Test set results

In [ ]:
from matplotlib.colors import ListedColormap
X_set, y_set = sc.inverse_transform(X_test), y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 10, stop = X_set[:, 0].max() + 10, step = 1),
                    np.arange(start = X_set[:, 1].min() - 1000, stop = X_set[:, 1].max() + 1000, step = 1))
plt.contourf(X1, X2, classifier.predict(sc.transform(np.array([X1.ravel(), X2.ravel()]).T)).reshape(X1.shape),
            alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1], c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Logistic Regression (Test set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()